In [ ]:
import sys
sys.path.append('/content')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
from __future__ import division
import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
np.random.seed(0)
import numpy as np
import string
import numpy as np
import pandas as pd
from keras.models import Model
import os
import math
from termcolor import colored
import numpy as np
import matplotlib
matplotlib.use('Agg')
import pandas as pd
#from data_helpers import load_data
from keras import callbacks
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
#from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from keras import backend as K
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D


In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/1000_tweets_per_user_train.csv')

In [ ]:
k=0
X_train=[]
y_train=[]
for i in range(0,50):
  for j in range(k,k+45):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=60

In [ ]:
k=45
X_test=[]
y_test=[]
for i in range(0,50):
  for j in range(k,k+5):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=60

In [ ]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

In [ ]:
train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

In [ ]:
np.random.shuffle(train)
np.random.shuffle(test)

In [ ]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower()
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>#@')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(rem_num)  
  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  stem_words=[stemmer.stem(w) for w in filtered_words]
  lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return " ".join(filtered_words)


train.iloc[:,0]=train.iloc[:,0].map(lambda s:preprocess(s))
test.iloc[:,0]=test.iloc[:,0].map(lambda s:preprocess(s))

In [ ]:
X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [ ]:
def create_vocab_set():
    alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                list(string.punctuation) + ['\n'] + [' '] )
    vocab_size = len(alphabet)
    check = set(alphabet)

    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabet):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, check

In [ ]:
maxlen = 50
vocab, reverse_vocab, vocab_size, check = create_vocab_set()

In [ ]:
def encode_data(x, maxlen, vocab, vocab_size, check):

    input_data = np.zeros((len(x), maxlen, vocab_size))
    for dix, sent in enumerate(x):
        counter = 0
        sent_array = np.zeros((maxlen, vocab_size))
        chars = list(sent.lower())
        for c in chars:
            if counter >= maxlen:
                pass
            else:
                char_array = np.zeros(vocab_size, dtype=np.int)
                if c in check:
                    ix = vocab[c]
                    char_array[ix] = 1
                sent_array[counter, :] = char_array
                counter += 1
        input_data[dix, :, :] = sent_array

    return input_data

In [ ]:
X_train = encode_data(X_train, maxlen, vocab, vocab_size, check)

In [ ]:
X_test = encode_data(X_test, maxlen, vocab, vocab_size, check)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X_train,y_train, test_size = 0.05, random_state = 0)

In [ ]:
def model(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter,
          cat_output):
    inputs = Input(shape=(maxlen, vocab_size), name='input', dtype='float32')

    conv = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[0],
                         border_mode='valid', activation='relu',
                         input_shape=(maxlen, vocab_size))(inputs)
    conv = MaxPooling1D(pool_length=3)(conv)

    conv1 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[1],
                          border_mode='valid', activation='relu')(conv)
    conv1 = MaxPooling1D(pool_length=3)(conv1)

    conv2 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[2],
                          border_mode='valid', activation='relu')(conv1)

    # conv3 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[3],
    #                       border_mode='valid', activation='relu')(conv2)

    # conv4 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[4],
    #                       border_mode='valid', activation='relu')(conv3)

    # conv5 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[5],---------
    #                       border_mode='valid', activation='relu')(conv4)
    # conv5 = MaxPooling1D(pool_length=3)(conv5)
    conv5 = Flatten()(conv2)

    #Two dense layers with dropout of .5
    z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv5))
    # z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(z))

    pred = Dense(cat_output, activation='softmax', name='output')(z)

    model = Model(input=inputs, output=pred)

    sgd = SGD(lr=0.01, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=sgd,
                  metrics=['accuracy'])

    return model

In [ ]:
    nb_filter = 450
    dense_outputs = 256
    filter_kernels = [1, 2, 3]
    cat_output = 50
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    model = model(filter_kernels, dense_outputs, 50, vocab_size, nb_filter, cat_output)
    model.summary()
    

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 50, 70)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 50, 450)           31950     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 16, 450)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 15, 450)           405450    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 5, 450)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 3, 450)            607950    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1350)              0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(50, 70), filters=450, kernel_size=1, padding="valid")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=450, kernel_size=2, padding="valid")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=450, kernel_s

In [ ]:
    Adam(lr=1.00, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
model.fit(np.array(X_train), np.array(y_train), validation_data=[np.array(X_eval), np.array(y_eval)],batch_size=32,epochs=15, verbose=1, callbacks=[early_stopping])

Train on 2137 samples, validate on 113 samples
Epoch 1/15
2137/2137 [==============================] - 1s 364us/step - loss: 3.9224 - accuracy: 0.0159 - val_loss: 3.9140 - val_accuracy: 0.0088
Epoch 2/15
2137/2137 [==============================] - 0s 185us/step - loss: 3.9135 - accuracy: 0.0225 - val_loss: 3.9139 - val_accuracy: 0.0177
Epoch 3/15
2137/2137 [==============================] - 0s 192us/step - loss: 3.9062 - accuracy: 0.0262 - val_loss: 3.9029 - val_accuracy: 0.0265
Epoch 4/15
2137/2137 [==============================] - 0s 192us/step - loss: 3.8716 - accuracy: 0.0346 - val_loss: 3.8878 - val_accuracy: 0.0354
Epoch 5/15
2137/2137 [==============================] - 0s 181us/step - loss: 3.8592 - accuracy: 0.0360 - val_loss: 3.8832 - val_accuracy: 0.0354
Epoch 6/15
2137/2137 [==============================] - 0s 193us/step - loss: 3.8372 - accuracy: 0.0440 - val_loss: 3.8852 - val_accuracy: 0.0354
Epoch 7/15
2137/2137 [==============================] - 0s 186us/step - loss:

In [ ]:
y_pred = model.predict(np.array(X_test))

In [ ]:
y_pred

array([[0.00123343, 0.02901449, 0.01839048, ..., 0.00506606, 0.01445343,
        0.02150319],
       [0.00190308, 0.02762362, 0.00829851, ..., 0.00856693, 0.05309406,
        0.01701066],
       [0.01080716, 0.03386877, 0.00304045, ..., 0.02344253, 0.01555482,
        0.01889009],
       ...,
       [0.00484491, 0.00998049, 0.00386896, ..., 0.007606  , 0.02594477,
        0.03630941],
       [0.01016528, 0.01006909, 0.00534534, ..., 0.00889059, 0.02079046,
        0.05381718],
       [0.00033147, 0.03398509, 0.01233465, ..., 0.00364451, 0.00721719,
        0.01219828]], dtype=float32)

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

In [ ]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

In [ ]:
result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

In [ ]:
predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

In [ ]:
print(result)
print(predicted)

[2, 31, 24, 35, 37, 12, 45, 19, 0, 32, 23, 5, 16, 27, 45, 11, 13, 30, 47, 30, 27, 30, 23, 15, 7, 35, 17, 37, 29, 17, 31, 48, 15, 26, 36, 17, 27, 22, 32, 25, 11, 7, 38, 23, 16, 33, 23, 4, 36, 2, 48, 1, 40, 39, 34, 21, 45, 34, 18, 32, 32, 18, 41, 6, 48, 19, 11, 41, 43, 38, 15, 29, 30, 35, 42, 46, 8, 21, 40, 3, 15, 1, 12, 24, 27, 39, 23, 39, 20, 2, 47, 9, 9, 12, 29, 46, 44, 27, 44, 25, 28, 20, 6, 7, 31, 43, 20, 16, 5, 6, 8, 17, 42, 10, 33, 14, 13, 49, 1, 46, 22, 2, 13, 42, 8, 44, 25, 34, 41, 47, 0, 0, 11, 3, 28, 36, 45, 38, 42, 12, 13, 43, 6, 26, 24, 35, 29, 18, 24, 43, 10, 43, 16, 38, 44, 49, 13, 8, 39, 40, 31, 29, 14, 16, 49, 41, 0, 12, 5, 37, 7, 47, 4, 49, 14, 30, 46, 41, 35, 46, 20, 14, 9, 38, 10, 25, 10, 26, 22, 44, 3, 33, 28, 21, 11, 34, 9, 4, 48, 21, 19, 6, 3, 25, 37, 42, 1, 21, 22, 40, 1, 8, 10, 19, 4, 15, 17, 47, 39, 7, 37, 48, 24, 5, 20, 5, 36, 31, 36, 9, 2, 4, 26, 19, 28, 18, 26, 0, 33, 28, 3, 18, 14, 32, 45, 34, 33, 22, 40, 49]
[24, 18, 18, 48, 2, 3, 26, 28, 21, 28, 47, 13, 44

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

cm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted)) 

Confusion Matrix :
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [1 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Accuracy Score : 0.084
Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.11      0.20      0.14         5
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         5
           9       0.14      0.20      0.17         5
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         5
          12       0.17      0.20      0.18         5
          13       0.00      0.00      0.00         5
          14     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
